In [1]:
import os
import geopandas as gpd
import gdal
import numpy as np
import numba
# import glob
from multiprocessing import Pool
import tqdm.auto as tqdm

In [2]:
tile_size_px = [1000,1000]
tile_oxerlap_px = 150
output_downsample = 0.5
output_folder = r''
geotiff_folder = r'/media/nick/2TB Working 2/Projects/GeoTIFF_tiler_data/VIVID_Landgate_20190910_112102323 (copy)'
input_file_ext = '.tif'

In [3]:
%%time
# search folder and all sub folders for '.tif' files
geo_tiff_list = []
for root, dirs, files in os.walk(geotiff_folder):
    for file in files:
        if file.endswith(input_file_ext):
            geo_tiff_list.append(os.path.join(root, file))
            
len(geo_tiff_list) 

CPU times: user 1.81 ms, sys: 0 ns, total: 1.81 ms
Wall time: 1.09 ms


272

In [4]:
def get_bounds(tif_path):
#     open file
    data = gdal.Open(tif_path)
#     grab bounds
    geoTransform = data.GetGeoTransform()
    left = geoTransform[0]
    top = geoTransform[3]
    right = left + geoTransform[1] * data.RasterXSize
    bottom = top + geoTransform[5] * data.RasterYSize
    return[top, left, bottom, right]

In [5]:
%%time
# use multiprocessing to extract raster bounds
# interesting when using a small number of geotiffs its is slightly quicker to just run this as a loop
# however once you get over a 100 or so this method is much quicker
with Pool() as pool:
    geo_tiff_bounds = list(tqdm.tqdm(pool.imap(get_bounds, geo_tiff_list), total=len(geo_tiff_list)))
# convert list to np array
geo_tiff_bounds = np.array(geo_tiff_bounds)
# grab max extents
top_max = geo_tiff_bounds[:,0].max()
left_max = geo_tiff_bounds[:,1].min()
bottom_max = geo_tiff_bounds[:,2].min()
right_max = geo_tiff_bounds[:,3].max()

  0%|          | 0/272 [00:00<?, ?it/s]

CPU times: user 35.7 ms, sys: 38.2 ms, total: 73.9 ms
Wall time: 150 ms
